# 1. Preprocessing data from the wikipage

In [2]:
import wikipedia
from bs4 import BeautifulSoup
import pandas as pd
import numpy as np

In [3]:
wiki = wikipedia.page("List of postal codes of Canada: M").html()
soup = BeautifulSoup(wiki)
soup

<html><body><div class="mw-parser-output"><p>This is a list of <a href="/wiki/Postal_codes_in_Canada" title="Postal codes in Canada">postal codes in Canada</a> where the first letter is M. Postal codes beginning with M are located within the city of <a href="/wiki/Toronto" title="Toronto">Toronto</a> in the province of <a href="/wiki/Ontario" title="Ontario">Ontario</a>. Only the first three characters are listed, corresponding to the Forward Sortation Area.
</p><p><a href="/wiki/Canada_Post" title="Canada Post">Canada Post</a> provides a free postal code look-up tool on its website,<sup class="reference" id="cite_ref-1"><a href="#cite_note-1">[1]</a></sup> via its <a href="/wiki/Mobile_app" title="Mobile app">applications</a> for such <a class="mw-redirect" href="/wiki/Smartphones" title="Smartphones">smartphones</a> as the <a href="/wiki/IPhone" title="IPhone">iPhone</a> and <a href="/wiki/BlackBerry" title="BlackBerry">BlackBerry</a>,<sup class="reference" id="cite_ref-2"><a href="#

### Extract table html using BeautifulSoup

In [25]:
table = soup.find("table")
table

<table class="wikitable sortable">
<tbody><tr>
<th>Postcode</th>
<th>Borough</th>
<th>Neighbourhood
</th></tr>
<tr>
<td>M1A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M2A</td>
<td>Not assigned</td>
<td>Not assigned
</td></tr>
<tr>
<td>M3A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Parkwoods" title="Parkwoods">Parkwoods</a>
</td></tr>
<tr>
<td>M4A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Victoria_Village" title="Victoria Village">Victoria Village</a>
</td></tr>
<tr>
<td>M5A</td>
<td><a href="/wiki/Downtown_Toronto" title="Downtown Toronto">Downtown Toronto</a></td>
<td><a href="/wiki/Regent_Park" title="Regent Park">Harbourfront</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North_York" title="North York">North York</a></td>
<td><a href="/wiki/Lawrence_Heights" title="Lawrence Heights">Lawrence Heights</a>
</td></tr>
<tr>
<td>M6A</td>
<td><a href="/wiki/North

In [5]:
columns = [th.get_text() for th in table.find("tr").find_all("th")]

In [6]:
columns[2] = "Neighbourhood"
columns

['Postcode', 'Borough', 'Neighbourhood']

In [7]:
dataset = []
for row in table.find_all("tr")[1:]:
    data = dict(zip(columns, (td.get_text() for td in row.find_all("td"))))
    data["Neighbourhood"]= data["Neighbourhood"].rstrip()
    dataset.append(data)
print(dataset)

[{'Postcode': 'M1A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned'}, {'Postcode': 'M2A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned'}, {'Postcode': 'M3A', 'Borough': 'North York', 'Neighbourhood': 'Parkwoods'}, {'Postcode': 'M4A', 'Borough': 'North York', 'Neighbourhood': 'Victoria Village'}, {'Postcode': 'M5A', 'Borough': 'Downtown Toronto', 'Neighbourhood': 'Harbourfront'}, {'Postcode': 'M6A', 'Borough': 'North York', 'Neighbourhood': 'Lawrence Heights'}, {'Postcode': 'M6A', 'Borough': 'North York', 'Neighbourhood': 'Lawrence Manor'}, {'Postcode': 'M7A', 'Borough': "Queen's Park", 'Neighbourhood': 'Not assigned'}, {'Postcode': 'M8A', 'Borough': 'Not assigned', 'Neighbourhood': 'Not assigned'}, {'Postcode': 'M9A', 'Borough': 'Downtown Toronto', 'Neighbourhood': "Queen's Park"}, {'Postcode': 'M1B', 'Borough': 'Scarborough', 'Neighbourhood': 'Rouge'}, {'Postcode': 'M1B', 'Borough': 'Scarborough', 'Neighbourhood': 'Malvern'}, {'Postcode': 'M2B', 'Borough': 'No

### Dataframe will consist of three columns: PostalCode, Borough, and Neighborhood and its values

In [8]:
df = pd.DataFrame(dataset)
print(df.shape)
print("**********************")
df.head()

(287, 3)
**********************


,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Only processing the cells that have an assigned borough

In [15]:
df = df[df["Borough"] != "Not assigned"].reset_index(drop=True)
print("After removing cells that have a borough as Not assigned:", df.shape)
print("\n**********************\n")
print("Preprocessed dataframe:\n\n",df.head())
print("\n **********************\n")
print("Verifying samples with common postal code and borough:\n\n", df[(df["Borough"] == "Etobicoke") & (df["Postcode"]=="M9V")])
print("\n**********************\n")
print("Neighbourhood that are with Not assigned:\n\n", df[df["Neighbourhood"] == "Not assigned"])

After removing cells that have a borough as Not assigned: (210, 3)

**********************

Preprocessed dataframe:

   Postcode           Borough     Neighbourhood
0      M3A        North York         Parkwoods
1      M4A        North York  Victoria Village
2      M5A  Downtown Toronto      Harbourfront
3      M6A        North York  Lawrence Heights
4      M6A        North York    Lawrence Manor

 **********************

Verifying samples with common postal code and borough:

     Postcode    Borough     Neighbourhood
173      M9V  Etobicoke    Albion Gardens
174      M9V  Etobicoke  Beaumond Heights
175      M9V  Etobicoke        Humbergate
176      M9V  Etobicoke         Jamestown
177      M9V  Etobicoke       Mount Olive
178      M9V  Etobicoke       Silverstone
179      M9V  Etobicoke     South Steeles
180      M9V  Etobicoke       Thistletown

**********************

Neighbourhood that are with Not assigned:

   Postcode       Borough Neighbourhood
5      M7A  Queen's Park  Not a

### Combing neighborhoods with a comma separated which are having same postal code area.

In [17]:
combi_neigh = df.groupby(["Postcode","Borough"])["Neighbourhood"].agg([('Neighbourhood', ', '.join)])
combi_neigh.reset_index(inplace = True)
combi_neigh

,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


### Neighbourhood that are with a Not assigned rows

In [20]:
combi_neigh[combi_neigh["Neighbourhood"]=="Not assigned"]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Not assigned


### Neighbourhood that are with a Not assigned indexes

In [21]:
combi_neigh.loc[combi_neigh["Neighbourhood"]=="Not assigned", "Borough"].index

Int64Index([85], dtype='int64')

### Updating neighbourhood that are with a Not assigned value  same as the borough value

In [22]:
# creating a list of borough values based on Neighbourhood column with a Not assigned value
borough_values= combi_neigh.loc[combi_neigh["Neighbourhood"]=="Not assigned", "Borough"].to_list()

# Extracing a list indexes of Neighbourhood column that are with a Not assigned
borough_indexes= combi_neigh.loc[combi_neigh["Neighbourhood"]=="Not assigned", "Borough"].index

print(borough_indexes)
print(borough_values)

def update_na_negigh(dataframe, keys,values ):
    for idx, key in enumerate(keys):
        dataframe.loc[key,'Neighbourhood'] = values[idx]
    return dataframe  

combi_neigh = update_na_negigh(dataframe=combi_neigh, keys=borough_indexes, values=borough_values)
combi_neigh

Int64Index([85], dtype='int64')
["Queen's Park"]


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae
...,...,...,...
98,M9N,York,Weston
99,M9P,Etobicoke,Westmount
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv..."
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ..."


### Verifying rows that are having same borough and neighbourhood values

In [23]:
combi_neigh.loc[combi_neigh["Neighbourhood"]==combi_neigh["Borough"]]

,Postcode,Borough,Neighbourhood
85,M7A,Queen's Park,Queen's Park


### The number of rows of in a preprocessed dataframe

In [26]:
combi_neigh.shape

(103, 3)

#  2. Getting the latitude and the longitude coordinates of each neighborhood

In [32]:
import geocoder # import geocoder

# initialize your variable to None
lat_lng_coords = None

# loop until you get the coordinates
# while(lat_lng_coords is None):
g = geocoder.google('{}, Toronto, Ontario'.format('M5G'))
lat_lng_coords = g.latlng

print(latitude = lat_lng_coords[0])
print(longitude = lat_lng_coords[1])

In [37]:
gps = pd.read_csv("Geospatial_Coordinates.csv")

In [34]:
combi_neigh.set_index("Postcode", inplace=True)
gps.set_index("Postal Code", inplace=True)

### Apply join operation to map lat, lng coordinates with preprocessed df by index matching.

In [38]:
df_gps = combi_neigh.join(gps)

In [40]:
df_gps.reset_index(inplace=True)

In [41]:
df_gps

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M1B,Scarborough,"Rouge, Malvern",43.806686,-79.194353
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union",43.784535,-79.160497
2,M1E,Scarborough,"Guildwood, Morningside, West Hill",43.763573,-79.188711
3,M1G,Scarborough,Woburn,43.770992,-79.216917
4,M1H,Scarborough,Cedarbrae,43.773136,-79.239476
...,...,...,...,...,...
98,M9N,York,Weston,43.706876,-79.518188
99,M9P,Etobicoke,Westmount,43.696319,-79.532242
100,M9R,Etobicoke,"Kingsview Village, Martin Grove Gardens, Richv...",43.688905,-79.554724
101,M9V,Etobicoke,"Albion Gardens, Beaumond Heights, Humbergate, ...",43.739416,-79.588437


# 3. Exploring and clustering the neighborhoods in Toronto

In [80]:
df_gps.Borough.to_list()

['Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'Scarborough',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'North York',
 'East York',
 'East York',
 'East Toronto',
 'East York',
 'East York',
 'East York',
 'East Toronto',
 'East Toronto',
 'East Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Central Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown Toronto',
 'Downtown 

### Extracting indexes of Borough values that are containing Toronto in it.

In [81]:
indxes_toronto = []
for ind, borugh in enumerate(df_gps.Borough.to_list()):
    if "Toronto" in borugh:
        indxes_toronto.append(ind)

In [87]:
# creating dataframe that are having toronto init.
toronto_df  = df_gps[df_gps.index.isin(indxes_toronto)]
toronto_df = toronto_df.reset_index(drop=True)

In [90]:
toronto_df

,Postcode,Borough,Neighbourhood,Latitude,Longitude
0,M4E,East Toronto,The Beaches,43.676357,-79.293031
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572
3,M4M,East Toronto,Studio District,43.659526,-79.340923
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790
5,M4P,Central Toronto,Davisville North,43.712751,-79.390197
6,M4R,Central Toronto,North Toronto West,43.715383,-79.405678
7,M4S,Central Toronto,Davisville,43.704324,-79.388790
8,M4T,Central Toronto,"Moore Park, Summerhill East",43.689574,-79.383160
9,M4V,Central Toronto,"Deer Park, Forest Hill SE, Rathnelly, South Hi...",43.686412,-79.400049


In [94]:
import folium
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import matplotlib.cm as cm
import matplotlib.colors as colors
import requests

# import k-means from clustering stage
from sklearn.cluster import KMeans
address = 'Toronto'

geolocator = Nominatim(user_agent="ny_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Toronto are {}, {}.'.format(latitude, longitude))

The geograpical coordinate of Toronto are 43.653963, -79.387207.


In [95]:
# create map of toronto using latitude and longitude values
map_toronto = folium.Map(location=[latitude, longitude], zoom_start=11)

# add markers to map
for lat, lng, label in zip(toronto_df['Latitude'], toronto_df['Longitude'], toronto_df['Neighbourhood']):
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_toronto)  
    
map_toronto

In [267]:
CLIENT_ID = 'xxxx' # your Foursquare ID
CLIENT_SECRET = 'xxxx' # your Foursquare Secret
VERSION = 'xxxx' # Foursquare API version

# print('Your credentails:')
# print('CLIENT_ID: ' + CLIENT_ID)
# print('CLIENT_SECRET:' + CLIENT_SECRET)

In [88]:
toronto_df['Latitude'][0]

43.67635739999999

In [89]:
neighborhood_latitude = toronto_df.loc[ 0, 'Latitude'] # neighborhood latitude value
neighborhood_longitude = toronto_df.loc[0, 'Longitude'] # neighborhood longitude value

neighborhood_name = toronto_df.loc[0, 'Neighbourhood'] # neighborhood name

print('Latitude and longitude values of {} are {}, {}.'.format(neighborhood_name, 
                                                               neighborhood_latitude, 
                                                               neighborhood_longitude))

Latitude and longitude values of The Beaches are 43.67635739999999, -79.2930312.


## Explore Neighborhoods in Toronto

In [96]:
LIMIT = 100
radius = 500 # define radius


def getNearbyVenues(names, latitudes, longitudes, radius=500):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            LIMIT)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

##### Now write the code to run the above function on each neighborhood and create a new dataframe called manhattan_venues.¶

In [97]:
toronto_venues = getNearbyVenues(names=toronto_df['Neighbourhood'],
                                   latitudes=toronto_df['Latitude'],
                                   longitudes=toronto_df['Longitude']
                                  )

The Beaches
The Danforth West, Riverdale
The Beaches West, India Bazaar
Studio District
Lawrence Park
Davisville North
North Toronto West
Davisville
Moore Park, Summerhill East
Deer Park, Forest Hill SE, Rathnelly, South Hill, Summerhill West
Rosedale
Cabbagetown, St. James Town
Church and Wellesley
Harbourfront
Ryerson, Garden District
St. James Town
Berczy Park
Central Bay Street
Adelaide, King, Richmond
Harbourfront East, Toronto Islands, Union Station
Design Exchange, Toronto Dominion Centre
Commerce Court, Victoria Hotel
Roselawn
Forest Hill North, Forest Hill West
The Annex, North Midtown, Yorkville
Harbord, University of Toronto
Chinatown, Grange Park, Kensington Market
CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara
Stn A PO Boxes 25 The Esplanade
First Canadian Place, Underground city
Christie
Dovercourt Village, Dufferin
Little Portugal, Trinity
Brockton, Exhibition Place, Parkdale Village
High Park, The Junction Sout

In [139]:
# Let's check the size of the resulting dataframe

print(toronto_venues.shape)
toronto_venues.head()

(1685, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,The Beaches,43.676357,-79.293031,Glen Manor Ravine,43.676821,-79.293942,Trail
1,The Beaches,43.676357,-79.293031,The Big Carrot Natural Food Market,43.678879,-79.297734,Health Food Store
2,The Beaches,43.676357,-79.293031,Grover Pub and Grub,43.679181,-79.297215,Pub
3,The Beaches,43.676357,-79.293031,Glen Stewart Ravine,43.676300,-79.294784,Other Great Outdoors
4,The Beaches,43.676357,-79.293031,Upper Beaches,43.680563,-79.292869,Neighborhood


##### Let's check how many venues were returned for each neighborhood

In [140]:
toronto_venues.groupby("Neighborhood").count()

,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
Neighborhood,,,,,,
"Adelaide, King, Richmond",100,100,100,100,100,100
Berczy Park,55,55,55,55,55,55
"Brockton, Exhibition Place, Parkdale Village",22,22,22,22,22,22
Business Reply Mail Processing Centre 969 Eastern,16,16,16,16,16,16
"CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara",15,15,15,15,15,15
"Cabbagetown, St. James Town",43,43,43,43,43,43
Central Bay Street,84,84,84,84,84,84
"Chinatown, Grange Park, Kensington Market",94,94,94,94,94,94
Christie,17,17,17,17,17,17


#### Let's find out how many unique categories can be curated from all the returned venues

In [146]:
print('There are {} uniques categories.'.format(len(toronto_venues['Venue Category'].unique())))
print(toronto_venues['Venue Category'].unique())

There are 232 uniques categories.
['Trail' 'Health Food Store' 'Pub' 'Other Great Outdoors' 'Neighborhood'
 'Greek Restaurant' 'Cosmetics Shop' 'Ice Cream Shop' 'Italian Restaurant'
 'Brewery' 'Yoga Studio' 'Fruit & Vegetable Store' 'Pizza Place'
 'Juice Bar' 'Bookstore' 'Restaurant' 'Dessert Shop' 'Bubble Tea Shop'
 'Spa' 'Furniture / Home Store' 'Diner' 'Coffee Shop' 'Grocery Store'
 'Caribbean Restaurant' 'Bakery' 'Liquor Store' 'American Restaurant'
 'Fish & Chips Shop' 'Gym' 'Burger Joint' 'Sushi Restaurant' 'Park'
 'Pet Store' 'Steakhouse' 'Burrito Place' 'Fast Food Restaurant'
 'Movie Theater' 'Sandwich Place' 'Board Shop' 'Food & Drink Shop'
 'Fish Market' 'Café' 'Seafood Restaurant' 'Thai Restaurant' 'Gay Bar'
 'Cheese Shop' 'Middle Eastern Restaurant' 'Comfort Food Restaurant'
 'Stationery Store' 'Coworking Space' 'Bar' 'Latin American Restaurant'
 'Gastropub' 'Bank' 'Gym / Fitness Center' 'Convenience Store'
 'Clothing Store' 'Swim School' 'Bus Line' 'Breakfast Spot' 'Hotel'

## Analyze Each Neighborhood

In [153]:
# one hot encoding
toronto_onehot = pd.get_dummies(toronto_venues[['Venue Category']], prefix="", prefix_sep="")
toronto_onehot = toronto_onehot.rename(columns = {"Neighborhood":"Neighborhood_venue" })
# add neighborhood column back to dataframe
toronto_onehot = pd.concat([toronto_venues["Neighborhood"], toronto_onehot], axis=1)
toronto_onehot.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,1,0,0,0,0,0,0,0
1,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,The Beaches,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [154]:
# And let's examine the new dataframe size.
toronto_onehot.shape

(1685, 233)

##### Next, let's group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [158]:
toronto_grouped = toronto_onehot.groupby('Neighborhood').mean().reset_index()
toronto_grouped.head()

,Neighborhood,Afghan Restaurant,Airport,Airport Food Court,Airport Gate,Airport Lounge,Airport Service,Airport Terminal,American Restaurant,Antique Shop,...,Thrift / Vintage Store,Toy / Game Store,Trail,Train Station,Vegetarian / Vegan Restaurant,Video Game Store,Vietnamese Restaurant,Wine Bar,Wings Joint,Yoga Studio
0,"Adelaide, King, Richmond",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.02,0.0,...,0.0,0.0,0.0,0.0,0.020000,0.0,0.0,0.01,0.0,0.000000
1,Berczy Park,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.018182,0.0,0.0,0.00,0.0,0.000000
2,"Brockton, Exhibition Place, Parkdale Village",0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.045455
3,Business Reply Mail Processing Centre 969 Eastern,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000
4,"CN Tower, Bathurst Quay, Island airport, Harbo...",0.0,0.066667,0.066667,0.066667,0.133333,0.133333,0.133333,0.00,0.0,...,0.0,0.0,0.0,0.0,0.000000,0.0,0.0,0.00,0.0,0.000000


#### Let's confirm the new size

In [160]:
toronto_grouped.shape

(38, 233)

#### Let's print each neighborhood along with the top 5 most common venues

In [161]:
num_top_venues = 5

for hood in toronto_grouped['Neighborhood']:
    print("----"+hood+"----")
    temp = toronto_grouped[toronto_grouped['Neighborhood'] == hood].T.reset_index()
#     temp = manhattan_grouped[manhattan_grouped['Neighborhood'] == hood].reset_index()
#     print(temp)
    temp.columns = ['venue','freq']
    temp = temp.iloc[1:]
    temp['freq'] = temp['freq'].astype(float)
    temp = temp.round({'freq': 2})
    print(temp.sort_values('freq', ascending=False).reset_index(drop=True).head(num_top_venues))
    print('\n')

----Adelaide, King, Richmond----
             venue  freq
0      Coffee Shop  0.07
1             Café  0.05
2  Thai Restaurant  0.04
3       Steakhouse  0.04
4       Restaurant  0.03


----Berczy Park----
                venue  freq
0         Coffee Shop  0.09
1        Cocktail Bar  0.04
2          Steakhouse  0.04
3  Seafood Restaurant  0.04
4         Cheese Shop  0.04


----Brockton, Exhibition Place, Parkdale Village----
            venue  freq
0  Breakfast Spot  0.09
1            Café  0.09
2     Coffee Shop  0.09
3             Gym  0.05
4          Bakery  0.05


----Business Reply Mail Processing Centre 969 Eastern----
              venue  freq
0           Brewery  0.06
1     Burrito Place  0.06
2  Recording Studio  0.06
3            Garden  0.06
4     Garden Center  0.06


----CN Tower, Bathurst Quay, Island airport, Harbourfront West, King and Spadina, Railway Lands, South Niagara----
                 venue  freq
0       Airport Lounge  0.13
1      Airport Service  0.13
2     Ai

#### Let's put that into a pandas dataframe

#### First, let's write a function to sort the venues in descending order.


In [171]:
def return_most_common_venues(row, num_top_venues):
    row_categories = row.iloc[1:]
#   print(row_categories)
    row_categories_sorted = row_categories.sort_values(ascending=False)
    return row_categories_sorted.index.values[0:num_top_venues]

##### Now let's create the new dataframe and display the top 10 venues for each neighborhood.

In [2]:
num_top_venues = 10
indicators = ['st', 'nd', 'rd']

# create columns according to number of top venues
columns = ['Neighborhood']
for ind in np.arange(num_top_venues):
    try:
        columns.append('{}{} Most Common Venue'.format(ind+1, indicators[ind]))
    except:
        columns.append('{}th Most Common Venue'.format(ind+1))

# create a new dataframe
neighborhoods_venues_sorted = pd.DataFrame(columns=columns)
neighborhoods_venues_sorted['Neighborhood'] = toronto_grouped['Neighborhood']

for ind in np.arange(toronto_grouped.shape[0]):
    neighborhoods_venues_sorted.iloc[ind, 1:] = return_most_common_venues(toronto_grouped.iloc[ind, :], num_top_venues)

# print(neighborhoods_venues_sorted)

neighborhoods_venues_sorted.head()

NameError: name 'pd' is not defined

## Cluster Neighborhoods

In [257]:
from sklearn.cluster import KMeans

kclusters = 4

toronto_grouped_clustering = toronto_grouped.drop('Neighborhood', 1)
# print(toronto_grouped_clustering)
# run k-means clustering
kmeans = KMeans(n_clusters=kclusters).fit(toronto_grouped_clustering)

# check cluster labels generated for each row in the dataframe
print(kmeans.labels_)
len(kmeans.labels_)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 0 1 2 1 1 0 3 1 1 1 1 1 1 1 1
 1]


38

##### Let's create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.

In [258]:
# add clustering labels
# neighborhoods_venues_sorted.insert(0, 'Cluster Labels', kmeans.labels_)
neighborhoods_venues_sorted['Cluster Labels'] = kmeans.labels_
toronto_merged = toronto_df

# merge toronto_grouped with toronto_data to add latitude/longitude for each neighborhood
toronto_merged = toronto_merged.join(neighborhoods_venues_sorted.set_index('Neighborhood'), on='Neighbourhood')
toronto_merged = toronto_merged.iloc[:-1,:]
toronto_merged['Cluster Labels'] = toronto_merged['Cluster Labels'].astype(int)
toronto_merged.head()# check the last columns!

,Postcode,Borough,Neighbourhood,Latitude,Longitude,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,M4E,East Toronto,The Beaches,43.676357,-79.293031,1,Other Great Outdoors,Trail,Pub,Neighborhood_venue,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
1,M4K,East Toronto,"The Danforth West, Riverdale",43.679557,-79.352188,1,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,M4L,East Toronto,"The Beaches West, India Bazaar",43.668999,-79.315572,1,Sandwich Place,Park,Board Shop,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Food & Drink Shop,Italian Restaurant
3,M4M,East Toronto,Studio District,43.659526,-79.340923,1,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Fish Market,Ice Cream Shop,Seafood Restaurant,Sandwich Place
4,M4N,Central Toronto,Lawrence Park,43.728020,-79.388790,0,Park,Gym / Fitness Center,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


### Let's visualize the resulting clusters

In [261]:
import numpy as np
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(toronto_merged['Latitude'], toronto_merged['Longitude'], toronto_merged['Neighbourhood'], toronto_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

## Examine Clusters

#### Cluster 1

In [262]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 0, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
4,Central Toronto,Park,Gym / Fitness Center,Swim School,Bus Line,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop
10,Downtown Toronto,Park,Playground,Trail,Dance Studio,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run
23,Central Toronto,Park,Jewelry Store,Trail,Sushi Restaurant,Yoga Studio,Department Store,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop


#### Cluster 2

In [263]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 1, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
0,East Toronto,Other Great Outdoors,Trail,Pub,Neighborhood_venue,Health Food Store,Doner Restaurant,Dog Run,Donut Shop,Dance Studio,Discount Store
1,East Toronto,Greek Restaurant,Coffee Shop,Italian Restaurant,Ice Cream Shop,Restaurant,Bookstore,Furniture / Home Store,Fruit & Vegetable Store,Pub,Pizza Place
2,East Toronto,Sandwich Place,Park,Board Shop,Ice Cream Shop,Fish & Chips Shop,Pub,Sushi Restaurant,Fast Food Restaurant,Food & Drink Shop,Italian Restaurant
3,East Toronto,Café,Coffee Shop,Bakery,Italian Restaurant,American Restaurant,Yoga Studio,Fish Market,Ice Cream Shop,Seafood Restaurant,Sandwich Place
5,Central Toronto,Hotel,Park,Clothing Store,Sandwich Place,Pizza Place,Food & Drink Shop,Breakfast Spot,Gym,Donut Shop,Doner Restaurant
6,Central Toronto,Clothing Store,Sporting Goods Shop,Coffee Shop,Yoga Studio,Gym / Fitness Center,Health & Beauty Service,Diner,Dessert Shop,Mexican Restaurant,Cosmetics Shop
7,Central Toronto,Sandwich Place,Dessert Shop,Pizza Place,Coffee Shop,Italian Restaurant,Gym,Café,Sushi Restaurant,Pharmacy,Costume Shop
9,Central Toronto,Pub,Coffee Shop,American Restaurant,Supermarket,Sushi Restaurant,Sports Bar,Pizza Place,Fried Chicken Joint,Bagel Shop,Restaurant
11,Downtown Toronto,Coffee Shop,Restaurant,Pub,Italian Restaurant,Bakery,Pizza Place,Flower Shop,Café,Grocery Store,Breakfast Spot
12,Downtown Toronto,Sushi Restaurant,Coffee Shop,Gay Bar,Japanese Restaurant,Restaurant,Gastropub,Café,Hotel,Mediterranean Restaurant,Gym


#### Cluster 3

In [264]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 2, toronto_merged.columns[[1] + list(range(6, toronto_merged.shape[1]))]]

,Borough,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
8,Central Toronto,Restaurant,Playground,Intersection,Deli / Bodega,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant,Dog Run


#### Cluster 4

In [266]:
toronto_merged.loc[toronto_merged['Cluster Labels'] == 3, toronto_merged.columns[[1] + list(range(5, toronto_merged.shape[1]))]]

,Borough,Cluster Labels,1st Most Common Venue,2nd Most Common Venue,3rd Most Common Venue,4th Most Common Venue,5th Most Common Venue,6th Most Common Venue,7th Most Common Venue,8th Most Common Venue,9th Most Common Venue,10th Most Common Venue
22,Central Toronto,3,Health & Beauty Service,Garden,Yoga Studio,Deli / Bodega,Ethiopian Restaurant,Electronics Store,Eastern European Restaurant,Dumpling Restaurant,Donut Shop,Doner Restaurant
